lightgbm

Average Log Loss after Full Training: 0.2319
Average Accuracy after Full Training: 0.9128
Average Precision after Full Training: 0.7076
Average Recall after Full Training: 0.9095
Average F1-score after Full Training: 0.7907

In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from tqdm.notebook import tqdm
# from sklearn.metrics import log_loss
# from sklearn.model_selection import StratifiedKFold
# from sklearn.impute import SimpleImputer
# from xgboost import XGBClassifier
# from sklearn import feature_selection


# def select_k_best_features(train):
#     # Define the target variable
#     target = train['Class']
    
#     # Drop unnecessary columns
#     train = train.drop(['Id', 'Class'], axis=1)
    
#     # Map 'EJ' column to binary values
#     train['EJ'] = train['EJ'].map({'A': 0, 'B': 1})
    
#     # Define the imputer
# #     imputer = SimpleImputer(strategy='mean')
#     imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    
#     # Apply mean imputation on the data
#     train = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)
    
#     # Define the threshold and t-score values
#     threshold = 0.01
#     t_score = 5
    
#     # Perform feature selection using SelectKBest and f_classif score function
#     fs = feature_selection.SelectKBest(score_func=feature_selection.f_classif, k=len(train.columns))
#     X_selected = fs.fit_transform(train, target.values)
    
#     print(f'Before the SelectKBest = {train.shape}')
    
#     new_features = []  # List of features with p-values below the threshold and scores above t_score
#     drop_features = []  # List of features with p-values above the threshold or scores below t_score
    
#     for i in range(len(train.columns)):
#         print('Feature {}: {:.3f} with p-value {:.3f}'.format(train.columns[i], fs.scores_[i], fs.pvalues_[i]))
#         if fs.pvalues_[i] <= threshold and fs.scores_[i] >= t_score:
#             new_features.append(train.columns[i])
#         else:
#             drop_features.append(train.columns[i])
    
#     X_selected_final = pd.DataFrame(X_selected)
#     X_selected_final.columns = train.columns
#     X_selected_final = X_selected_final[new_features]
    
#     print('=' * 30)
#     print('After the SelectKBest = {}'.format(X_selected_final.shape))
#     print('Drop-out Features = {}'.format(len(drop_features)))
    
#     return X_selected_final

# # Define the number of bags and folds
# bag_num = 5
# n_fold = 10

# # Define the competition log loss metric
# def competition_log_loss(y_true, y_pred):
#     N_0 = np.sum(1 - y_true)
#     N_1 = np.sum(y_true)
#     p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
#     p_0 = 1 - p_1
#     log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
#     log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
#     return (log_loss_0 + log_loss_1) / 2

# # Load the data
# COMP_PATH = "/kaggle/input/icr-identify-age-related-conditions"
# train = pd.read_csv(f"{COMP_PATH}/train.csv")
# test = pd.read_csv(f"{COMP_PATH}/test.csv")

# # train.drop(['EJ'],axis=1)

# # test.drop(['EJ'],axis=1)

# # Apply feature selection
# df = select_k_best_features(train)
# test_df = test[df.columns]

# # Define the XGBoost parameters
# xgb_params = {
#     "objective": "binary:logistic",
#     "learning_rate": 0.06733232950390658,
#     "n_estimators": 50000,
#     "early_stopping_rounds": 300,
#     "random_state": 118,
#     "subsample": 0.8,
#     "colsample_bytree": 0.6055755840633003,
#     "scale_pos_weight": 1 / np.mean(train['Class']),
#     "max_depth": 3
# }


# # Initialize lists to store models and log losses
# models = []
# bag_log_losses = []
# feature_importance_df_total = pd.DataFrame()
# df['Class'] = train['Class']

# # Perform bagging and feature selection
# for bag in range(bag_num):
#     print(f'########################## bag: {bag} ##########################')
#     kf = StratifiedKFold(n_splits=n_fold, random_state=2023 * bag, shuffle=True)
#     fold_losses = []
#     for fold, (train_idx, test_idx) in enumerate(kf.split(df, df['Class'])):
#         train_df = df.iloc[train_idx]
#         valid_df = df.iloc[test_idx]
#         valid_ids = valid_df.index.values

#         X_train, y_train = train_df.drop('Class', axis=1), train_df['Class']
#         X_valid, y_valid = valid_df.drop('Class', axis=1), valid_df['Class']

#         xgb = XGBClassifier(**xgb_params)
#         xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, eval_metric='logloss')

#         models.append(xgb)

#         # Calculate feature importances
#         feature_importances = xgb.feature_importances_
#         feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
#         feature_importance_df['Bag'] = bag
#         feature_importance_df['Fold'] = fold
#         feature_importance_df_total = pd.concat([feature_importance_df_total, feature_importance_df], axis=0)

#         y_pred = xgb.predict_proba(X_valid)
#         y_pred_postprocessed = np.where(y_pred[:, 1] > 0.8, 1, np.where(y_pred[:, 1] < 0.14, 0, y_pred[:, 1]))
#         fold_loss = log_loss(y_valid, y_pred_postprocessed)
#         fold_losses.append(fold_loss)
        
#         print(f"Total train: {len(train_df)}, Total valid: {len(valid_df)}, Bags: {bag}, Fold: {fold}, Log Loss: {fold_loss:.4f}")
    
#     avg_fold_loss = np.mean(fold_losses)
#     bag_log_losses.append(avg_fold_loss)
#     print(f"Average Log Loss for Bag {bag}: {avg_fold_loss:.4f}")

# avg_loss = np.mean(bag_log_losses)
# print(f"Average Log Loss after Full Training: {avg_loss:.4f}")

# # Calculate weights based on inverse of log loss
# weights = [1 / loss for loss in bag_log_losses]
# total_weight = sum(weights)
# weights = [weight / total_weight for weight in weights]

# # Prepare submission dataframe
# test_df_selected = test_df[X_train.columns]  # Apply the same feature selection to the test data
# preds = np.zeros(len(test_df_selected))
# for bag, weight in zip(range(bag_num), weights):
#     for fold in range(n_fold):
#         clf = models[bag * n_fold + fold]
#         preds += weight * clf.predict_proba(test_df_selected)[:, 1] / n_fold

# # Post-process predicted probabilities
# preds_postprocessed = np.where(preds > 0.8, 1, np.where(preds < 0.14, 0, preds))

# # Create submission dataframe
# submission = pd.DataFrame({'Id': test['Id'], 'Class_0': 1 - preds_postprocessed, 'Class_1': preds_postprocessed})
# submission.to_csv('submission.csv', index=False)
# submission.head()


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn.calibration import CalibratedClassifierCV

def select_k_best_features(train):
    # Define the target variable
    target = train['Class']
    
    # Drop unnecessary columns
    train = train.drop(['Id', 'Class'], axis=1)
    
    # Map 'EJ' column to binary values
    train['EJ'] = train['EJ'].map({'A': 0, 'B': 1})
    
    # Define the imputer
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    
    # Apply mean imputation on the data
    train = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)
    
    # Define the threshold and t-score values
    threshold = 0.01
    t_score = 5
    
    # Perform feature selection using SelectKBest and f_classif score function
    fs = feature_selection.SelectKBest(score_func=feature_selection.f_classif, k=len(train.columns))
    X_selected = fs.fit_transform(train, target.values)
    
    print(f'Before the SelectKBest = {train.shape}')
    
    new_features = []  # List of features with p-values below the threshold and scores above t_score
    drop_features = []  # List of features with p-values above the threshold or scores below t_score
    
    for i in range(len(train.columns)):
        print('Feature {}: {:.3f} with p-value {:.3f}'.format(train.columns[i], fs.scores_[i], fs.pvalues_[i]))
        if fs.pvalues_[i] <= threshold and fs.scores_[i] >= t_score:
            new_features.append(train.columns[i])
        else:
            drop_features.append(train.columns[i])
    
    X_selected_final = pd.DataFrame(X_selected)
    X_selected_final.columns = train.columns
    X_selected_final = X_selected_final[new_features]
    
    print('=' * 30)
    print('After the SelectKBest = {}'.format(X_selected_final.shape))
    print('Drop-out Features = {}'.format(len(drop_features)))
    
    return X_selected_final

# Define the number of bags and folds
bag_num = 5
n_fold = 10

# Define the competition log loss metric
def competition_log_loss(y_true, y_pred):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    return (log_loss_0 + log_loss_1) / 2

# Load the data
COMP_PATH = "/kaggle/input/icr-identify-age-related-conditions"
train = pd.read_csv(f"{COMP_PATH}/train.csv")
test = pd.read_csv(f"{COMP_PATH}/test.csv")

# Apply feature selection
df = select_k_best_features(train)
test_df = test[df.columns]

# Define the XGBoost parameters
xgb_params = {
    "objective": "binary:logistic",
    "learning_rate": 0.06733232950390658,
    "n_estimators": 50000,
    "early_stopping_rounds": 300,
    "random_state": 118,
    "subsample": 0.8,
    "colsample_bytree": 0.6055755840633003,
    "scale_pos_weight": 1 / np.mean(train['Class']),
    "max_depth": 3
}

# Initialize lists to store models and log losses
models = []
bag_log_losses = []
feature_importance_df_total = pd.DataFrame()
df['Class'] = train['Class']

# Perform bagging and feature selection
for bag in range(bag_num):
    print(f'########################## bag: {bag} ##########################')
    kf = StratifiedKFold(n_splits=n_fold, random_state=2023 * bag, shuffle=True)
    fold_losses = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(df, df['Class'])):
        train_df = df.iloc[train_idx]
        valid_df = df.iloc[test_idx]
        valid_ids = valid_df.index.values

        X_train, y_train = train_df.drop('Class', axis=1), train_df['Class']
        X_valid, y_valid = valid_df.drop('Class', axis=1), valid_df['Class']

        xgb = XGBClassifier(**xgb_params)
        xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, eval_metric='logloss')

        models.append(xgb)

        # Calibrate the classifier using isotonic regression
        calibrated_clf = CalibratedClassifierCV(xgb, method='isotonic', cv='prefit')
        calibrated_clf.fit(X_valid, y_valid)

        y_pred = calibrated_clf.predict_proba(X_valid)
        y_pred_reshaped = y_pred[:, 1]  # Select the probabilities of the positive class
        fold_loss = competition_log_loss(y_valid, y_pred_reshaped)
        fold_losses.append(fold_loss)
        
        print(f"Total train: {len(train_df)}, Total valid: {len(valid_df)}, Bags: {bag}, Fold: {fold}, Log Loss: {fold_loss:.4f}")
    
    avg_fold_loss = np.mean(fold_losses)
    bag_log_losses.append(avg_fold_loss)
    print(f"Average Log Loss for Bag {bag}: {avg_fold_loss:.4f}")

avg_loss = np.mean(bag_log_losses)
print(f"Average Log Loss after Full Training: {avg_loss:.4f}")

# Calculate class weights based on inverse of log loss
weights = [1 / loss for loss in bag_log_losses]
total_weight = sum(weights)
weights = [weight / total_weight for weight in weights]

# Prepare submission dataframe
test_df_selected = test_df[X_train.columns]  # Apply the same feature selection to the test data
preds = np.zeros(len(test_df_selected))
for bag, weight in zip(range(bag_num), weights):
    for fold in range(n_fold):
        clf = models[bag * n_fold + fold]
        calibrated_clf = CalibratedClassifierCV(clf, method='isotonic', cv='prefit')
        calibrated_clf.fit(X_valid, y_valid)
        fold_preds = calibrated_clf.predict_proba(test_df_selected)[:, 1]
        
        # Apply sigmoid function to adjust probabilities
        fold_preds = 1 / (1 + np.exp(-fold_preds))
     
        # Apply threshold for class 1
        fold_preds = np.where(fold_preds > 0.5, 1, 0)
        
        preds += weight * fold_preds / n_fold

# Create submission dataframe
submission = pd.DataFrame({'Id': test['Id'], 'Class_0': 1 - preds, 'Class_1': preds})
submission.to_csv('submission.csv', index=False)
submission.head()


Before the SelectKBest = (617, 56)
Feature AB: 52.566 with p-value 0.000
Feature AF: 62.007 with p-value 0.000
Feature AH: 1.228 with p-value 0.268
Feature AM: 37.314 with p-value 0.000
Feature AR: 2.560 with p-value 0.110
Feature AX: 5.917 with p-value 0.015
Feature AY: 4.206 with p-value 0.041
Feature AZ: 0.112 with p-value 0.738
Feature BC: 15.316 with p-value 0.000
Feature BD : 7.839 with p-value 0.005
Feature BN: 25.169 with p-value 0.000
Feature BP: 15.600 with p-value 0.000
Feature BQ: 58.146 with p-value 0.000
Feature BR: 4.861 with p-value 0.028
Feature BZ: 7.872 with p-value 0.005
Feature CB: 0.128 with p-value 0.720
Feature CC: 1.847 with p-value 0.175
Feature CD : 18.593 with p-value 0.000
Feature CF: 7.498 with p-value 0.006
Feature CH: 0.041 with p-value 0.840
Feature CL: 0.175 with p-value 0.676
Feature CR: 33.582 with p-value 0.000
Feature CS: 1.387 with p-value 0.239
Feature CU: 4.285 with p-value 0.039
Feature CW : 2.423 with p-value 0.120
Feature DA: 26.873 with p-va

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Total train: 555, Total valid: 62, Bags: 0, Fold: 0, Log Loss: 0.2580


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Total train: 555, Total valid: 62, Bags: 0, Fold: 1, Log Loss: 0.0000


...

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Total train: 556, Total valid: 61, Bags: 4, Fold: 8, Log Loss: 0.2058


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Total train: 556, Total valid: 61, Bags: 4, Fold: 9, Log Loss: 0.1695
Average Log Loss for Bag 4: 0.2144
Average Log Loss after Full Training: 0.1915


,Id,Class_0,Class_1
0,00eed32682bb,0.564472,0.435528
1,010ebe33f668,0.564472,0.435528
2,02fa521e1838,0.564472,0.435528
3,040e15f562a2,0.564472,0.435528
4,046e85c7cc7f,0.564472,0.435528


I've added calibration to the model predictions using isotonic regression to ensure better probability estimates